In [0]:

import math
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import skimage
import pickle as pkl
import cv2
import tensorflow.python.platform
from keras.preprocessing import sequence
from collections import Counter

In [50]:
from keras import backend as k 
k.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Give csv of the dataset for which feature extraction has to be performed
df_train = pd.read_csv('/content/drive/My Drive/code/VQAMed2018Train/VQAMed2018Train-QA.csv',sep='\t',header=None)
df_test = pd.read_csv('/content/drive/My Drive/code/VQAMed2018Valid/VQAMed2018Valid-QA.csv',sep='\t',header=None)
#df_test = pd.read_csv('/content/drive/My Drive/code/VQAMed2018Test/VQAMed2018Test-QA-Eval.csv',sep='\t',header=None)

In [53]:
df_train=df_train[[0,1,2]]
df_test=df_test[[0,1,2]]
df_train.head(5)

,0,1,2
0,1,rjv03401,what does mri show?
1,2,AIAN-14-313-g002,where does axial section mri abdomen show hypo...
2,3,wjem-11-76f3,what do the arrows denote in the noncontrast c...
3,4,ccr30002-0045-f3,what was normal?
4,5,rjt01904,what shows evidence of a contained rupture?


In [0]:
df_train.columns=['Serialno','imageno','question']
df_test.columns=['Serialno','imageno','question']

In [0]:
def removeSpan(df):
  boolmat=[]
  for i in range(0,len(df)):
      boolmat.append(df['question'].iloc[i][len(df['question'].iloc[i])-1]=='?')
  df=df[boolmat]

In [0]:
removeSpan(df_train)
#removeSpan(df_valid)
removeSpan(df_test)

In [59]:
len(df_train)
len(df_test)


500

Train Images

In [0]:
#path to VGG model 
vgg_path = '/content/drive/My Drive/code/model/vgg16-20160129.tfmodel'
#path to directory that contains images
img_path_train='/content/drive/My Drive/code/VQAMed2018Train/VQAMed2018Train-images'

def crop_image(x, target_height=227, target_width=227, as_float=True):
    image = cv2.imread(x)
    if as_float:
        image = image.astype(np.float32)

    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]

    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    return cv2.resize(resized_image, (target_height, target_width))

def read_image(path):
    img = crop_image(path, target_height=224, target_width=224)
    if img.shape[2] == 4:
        img = img[:,:,:3]
    img = img[None, ...]
    return img

tf.reset_default_graph()
with open(vgg_path,'rb') as f:
        fileContent = f.read()
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(fileContent)
final_array_train_total = []
images = tf.placeholder("float32", [1, 224, 224, 3])
tf.import_graph_def(graph_def, input_map={"images":images})
graph = tf.get_default_graph()

with tf.Session() as sess:    
    for i in df_train.imageno:
        feat = read_image(img_path_train+"/"+i+".jpg")
        #Here activations extracted from the last fully connected layers are used as features
        fc7 = sess.run(graph.get_tensor_by_name("import/Relu_1:0"), feed_dict={images:feat})
        fc7 = np.squeeze(fc7)
        final_array_train_total.append(fc7)
     

Validation Images

In [0]:
#path to VGG model 
vgg_path = '/content/drive/My Drive/code/model/vgg16-20160129.tfmodel'
#path to directory that contains images
img_path_test='/content/drive/My Drive/code/VQAMed2018Valid/VQAMed2018Valid-images'
def crop_image(x, target_height=227, target_width=227, as_float=True):
    image = cv2.imread(x)
    if as_float:
        image = image.astype(np.float32)

    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]

    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    return cv2.resize(resized_image, (target_height, target_width))

def read_image(path):
    img = crop_image(path, target_height=224, target_width=224)
    if img.shape[2] == 4:
        img = img[:,:,:3]
    img = img[None, ...]
    return img

tf.reset_default_graph()
with open(vgg_path,'rb') as f:
        fileContent = f.read()
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(fileContent)
final_array_test = []
images = tf.placeholder("float32", [1, 224, 224, 3])
tf.import_graph_def(graph_def, input_map={"images":images})
graph = tf.get_default_graph()
    
with tf.Session() as sess:    
    for i in df_test.imageno:
        feat = read_image(img_path_test+"/"+i+".jpg")
        #Here activations extracted from the last fully connected layers are used as features
        fc7 = sess.run(graph.get_tensor_by_name("import/Relu_1:0"), feed_dict={images:feat})      
        fc7=np.squeeze(fc7)
        final_array_test.append(fc7)



In [65]:
len(final_array_train_total)
len(final_array_test)

500

In [0]:
#Saving the features extracted for the images
import pickle
pickle.dump(final_array_train_total, open( "/content/drive/My Drive/code/image_feature_train.pkl", "wb" ) )

In [0]:
#Saving the features extracted for the images
import pickle
pickle.dump(final_array_test, open( "/content/drive/My Drive/code/image_feature_test.pkl", "wb" ) )